In [2]:
import pandas as pd

url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQ2G-ZFSotS2qo94mnTWCDvj9Y49ai-9O61DA7940sPYynEdBXq2cT2-Wl3nNldIb3gkpbPFaTFY2PJ/pub?output=xlsx"

xls = pd.ExcelFile(url)
xls

In [9]:
sheets = {
    sheet_name: pd.read_excel(xls, sheet_name)
    for sheet_name in xls.sheet_names
}
for sheet in sheets.values():
    sheet['grounding'] = sheet['grounding'].map(eval)
set(sheets)

{'BIOMD0000000955',
 'BIOMD0000000956',
 'BIOMD0000000957',
 'BIOMD0000000958',
 'BIOMD0000000960'}

In [12]:
from collections import defaultdict
ground_truth_df = pd.read_csv("/Users/cthoyt/Downloads/biomodels_groundings.tsv", sep='\t')

dd = defaultdict(set)
for model_id, curie in ground_truth_df.values:
    dd[model_id].add(curie)
dd = dict(dd)
dd

{'BIOMD0000000958': {'apollosv:00000154',
  'ido:0000511',
  'ido:0000514',
  'ido:0000592',
  'ncit:C25179',
  'ncit:C25376',
  'ncit:C28554',
  'ncit:C3833',
  'ncit:C49508'},
 'BIOMD0000000960': {'apollosv:00000154',
  'biomodels.species:BIOMD0000000960:Cumulative_Cases',
  'ido:0000480',
  'ido:0000511',
  'ido:0000514',
  'ido:0000592',
  'ncit:C25179',
  'ncit:C25269',
  'ncit:C28554',
  'ncit:C3833',
  'ncit:C68851'},
 'BIOMD0000000957': {'ido:0000511',
  'ido:0000514',
  'ido:0000592',
  'ncit:C15220'},
 'BIOMD0000000956': {'ido:0000511', 'ido:0000514', 'ido:0000592'},
 'BIOMD0000000955': {'ido:0000511',
  'ido:0000514',
  'ido:0000592',
  'ncit:C113725',
  'ncit:C15220',
  'ncit:C25269',
  'ncit:C25467',
  'ncit:C28554'}}

In [17]:
rows = []
for model_id, sheet in sheets.items():
    sheet_curies = {
        grounding["id"]
        for groundings in sheet["grounding"]
        for grounding in groundings
    }
    ground_truth_curies = dd[model_id]

    intersection_curies = sheet_curies & ground_truth_curies
    n_intersection = len(intersection_curies)
    n_ground_truth = len(ground_truth_curies)
    n_sheet = len(sheet_curies)
    rows.append((
        model_id,
        n_ground_truth,
        n_intersection,
        round(n_intersection / n_ground_truth, 2)
    ))

print("The way to interpret the precision is the percent")
pd.DataFrame(rows, columns=["model", "structured", "TA-1", "precision"])

,model,structured,TA-1,precision
0,BIOMD0000000955,8,1,0.12
1,BIOMD0000000956,3,0,0.00
2,BIOMD0000000957,4,1,0.25
3,BIOMD0000000958,9,1,0.11
4,BIOMD0000000960,11,1,0.09
